In [29]:
import subprocess
import os
import re
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

In [38]:
# --- 1. 配置 ---
SOURCE_DOC_FILE = "./yq2021-0112化解当下世界经济衰退的根本之道——基础设施建设.doc"

# Milvus 配置
USE_MILVUS_LITE = False  # 您正在使用 Docker 模式
MILVUS_HOST = "192.168.16.138" 
MILVUS_PORT = "19530"

COLLECTION_NAME = "report_analysis"
ID_FIELD = "chunk_id"
VECTOR_FIELD = "vector"
TEXT_FIELD = "text_content"
MODEL_NAME = 'all-mpnet-base-v2'
EMBEDDING_DIM = 768

MAX_CHUNK_LEN = 500  # 合并的目标上限
MIN_CHUNK_LEN = 100  # 合并的最小阈值
HARD_MAX_LEN = 1000 # 【新】单个段落的硬性拆分上限

In [31]:
# --- 2. 文档加载 ---
def find_libreoffice():
    possible_paths = [
        r"C:\Program Files\LibreOffice\program\soffice.exe",
        r"C:\Program Files (x86)\LibreOffice\program\soffice.exe",
        r"D:\LibreOffice\program\soffice.exe",
        "soffice"
    ]
    for path in possible_paths:
        try:
            if path != "soffice" and not os.path.exists(path): continue
            result = subprocess.run([path, '--version'], capture_output=True, text=True, timeout=10)
            if result.returncode == 0: return path
        except: continue
    return None

In [ ]:
def load_doc_as_text(doc_path):
    print("正在加载文档...")
    libreoffice_path = find_libreoffice()
    if not libreoffice_path:
        print("错误：未找到 LibreOffice (soffice.exe)，请检查路径。")
        return None
    
    doc_path = os.path.abspath(doc_path)
    output_dir = os.path.dirname(doc_path)
    html_filename = os.path.basename(doc_path).rsplit('.', 1)[0] + '.html'
    html_path = os.path.join(output_dir, html_filename)
    
    if os.path.exists(html_path): 
        try: os.remove(html_path)
        except: pass
        
    cmd = [libreoffice_path, '--headless', '--convert-to', 'html', '--outdir', output_dir, doc_path]
    subprocess.run(cmd, capture_output=True, text=True)
    
    if not os.path.exists(html_path):
        print(f"错误：LibreOffice 转换失败，未在 {output_dir} 找到 {html_filename}")
        return None
    
    content = ""
    for enc in ['utf-8', 'gb18030', 'gbk']:
        try:
            with open(html_path, 'r', encoding=enc) as f:
                content = f.read()
                break
        except: continue
            
    if not content:
        print("错误：读取转换后的 HTML 文件失败。")
        return None

    soup = BeautifulSoup(content, 'html.parser')
    
    # --- 移除页眉和页脚 ---
    # 在查找 <p> 标签之前，先把页眉和页脚的 div 整个移除掉
    print("   -> 正在移除页眉 (header) 和页脚 (footer)...")
    
    # 根据您提供的 .html 源码，页眉和页脚是 <div title="header"> 和 <div title="footer">
    header = soup.find('div', title='header')
    if header:
        header.decompose() # 移除页眉

    footer = soup.find('div', title='footer')
    if footer:
        footer.decompose() # 移除页脚
    
    # 1. 查找所有 *剩余的* <p> 标签
    paragraphs = soup.find_all('p')
    
    cleaned_paragraphs = []
    for p in paragraphs:
        # 2. 对每个 <p> 标签，获取其所有内部文本
        p_text = p.get_text().strip()
        
        # 3. 只有非空的段落才保留
        if p_text:
            cleaned_paragraphs.append(p_text)
    
    # 4. 用单一的、可靠的 \n 将所有“干净”的段落连接起来
    text = "\n".join(cleaned_paragraphs)
    
    try: 
        os.remove(html_path) # 恢复自动删除
        print("   -> 中间 HTML 文件已清理。")
    except: 
        pass
    
    return text

In [ ]:
# --- 3. 语义切割逻辑 (新 v3 - 增加硬上限) ---
import re

def split_long_paragraph(text, hard_max):
    """
    【新规则】: 将 > hard_max (1000) 的单个段落从“中间的句号”切分为两块。
    """
    print(f"   -> 检测到长段落 (长度 {len(text)})，尝试按中间句号 '。' 切分...")
    mid_point = len(text) // 2
    
    # 1. 寻找离中点最近的句号
    pos_before = text.rfind('。', 0, mid_point)
    pos_after = text.find('。', mid_point)
    
    split_pos = -1
    
    if pos_before != -1 and pos_after != -1:
        # 找到前后都有，取最近的
        if (mid_point - pos_before) < (pos_after - mid_point):
            split_pos = pos_before
        else:
            split_pos = pos_after
    elif pos_before != -1:
        # 只有前面有
        split_pos = pos_before
    elif pos_after != -1:
        # 只有后面有
        split_pos = pos_after
    else:
        # 整个段落没有句号
        print(f"   -> 警告: 段落长度 {len(text)} > {hard_max}，但未找到 '。' 无法切分。")
        return [text] # 无法切分，返回原样
    
    # 2. 执行切分 (句号保留在前一块)
    part1 = text[:split_pos + 1].strip()
    part2 = text[split_pos + 1:].strip()
    
    if not part1 or not part2:
         print(f"   -> 警告: 按句号切分失败 (产生空块)，返回原长段落。")
         return [text] # 避免切出空块
         
    print(f"   -> 成功切分为两块: (长度 {len(part1)}) 和 (长度 {len(part2)})")
    return [part1, part2]


def split_text_smart(text, max_len=MAX_CHUNK_LEN, min_len=MIN_CHUNK_LEN):
    """
    【新逻辑 v3】：
    1. 按段落提取。
    2. > 1000字的长段落按中间句号切分。
    3. 其余段落按 500 字上限合并。
    4. < 100字的小块与后续块合并。
    """
    print(f"正在按段落切割 (合并目标: {max_len}, 最小: {min_len}, 硬上限: {HARD_MAX_LEN})...")
    
    # 1. 移除废弃内容
    text = re.split(r"〖相关链接：信息〗|〖相关链接：报告〗", text, 1)[0]
    
    # 2. 基础清理 (保留了 v2 的修复)
    text = re.sub(r"\"", "", text)
    text = re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1f\x7f-\x9f]", "", text)
    text = re.sub(r"[\r\n\t ]+\n", "\n", text)
    text = re.sub(r"\n\s*\n", "\n", text).strip()
    
    paragraphs = text.split('\n')
    paragraphs = [p.strip() for p in paragraphs if p.strip()]

    # 3. 长度控制
    initial_chunks = []
    current_chunk = ""
    
    for para in paragraphs:
        
        # 3a. 检查单个段落是否超过硬上限 (1000)
        if len(para) > HARD_MAX_LEN:
            sub_paragraphs = split_long_paragraph(para, HARD_MAX_LEN)
        else:
            sub_paragraphs = [para]

        # 3b. 迭代处理(可能被切分后)的段落，并应用合并逻辑
        for sub_para in sub_paragraphs:
            if not sub_para.strip(): continue # 确保子段落不是空的
            
            # (应用 500 字的合并逻辑)
            if not current_chunk:
                current_chunk = sub_para
            elif len(current_chunk) + len(sub_para) + 1 <= max_len: 
                current_chunk += "\n" + sub_para
            else:
                initial_chunks.append(current_chunk)
                current_chunk = sub_para
    
    # 保存最后一个累积的块
    if current_chunk:
        initial_chunks.append(current_chunk)
        
    print(f"初步切割得到 {len(initial_chunks)} 个块，正在进行小块合并...")

    # 4. 后处理：合并小于 min_len (100) 的块
    final_chunks = []
    i = 0
    while i < len(initial_chunks):
        current_chunk = initial_chunks[i]
        
        if len(current_chunk) < min_len and (i < len(initial_chunks) - 1):
            next_chunk = initial_chunks[i+1]
            merged_chunk = current_chunk + "\n" + next_chunk
            
            if len(merged_chunk) <= max_len:
                final_chunks.append(merged_chunk)
                i += 2 
            else:
                final_chunks.append(current_chunk)
                i += 1
        else:
            final_chunks.append(current_chunk)
            i += 1

    print(f"清理完成，最终得到 {len(final_chunks)} 个文本块。")
    return final_chunks

In [44]:
# --- 4. 主程序 ---
def main():
    full_text = load_doc_as_text(SOURCE_DOC_FILE)
    if not full_text: 
        print("错误：未能加载文档。")
        return

    # 1. 执行切割
    chunks = split_text_smart(full_text)
    
    # 2. 预览检查
    print("\n" + "="*30)
    print("      最终切分效果预览      ")
    print("="*30)
    
    # 【修改点】简化预览逻辑，只看前 5 块和总块数
    for i, chunk in enumerate(chunks):
        if i < 5: # 只预览前5个
            print(f"\n>>> Chunk {i} (长度: {len(chunk)})")
            preview = chunk.replace("\n", " ")[:80]
            print(f"内容: {preview}...")
            
            # 再次校验
            if len(chunk) < MIN_CHUNK_LEN and i < len(chunks) - 1:
                # 这个警告是正常的，如果它无法与前后合并
                print(f"   -> 提示：块 {i} 长度为 {len(chunk)}，小于 {MIN_CHUNK_LEN}。")
    
    if len(chunks) > 5:
        print(f"\n... (及其他 {len(chunks) - 5} 个块)")

    # 3. 存入 Milvus
    print(f"\n正在生成向量...")
    model = SentenceTransformer(MODEL_NAME)
    embeddings = model.encode(chunks, show_progress_bar=True)
    
    print(f"正在连接 Milvus ({MILVUS_HOST})...")
    if USE_MILVUS_LITE:
        connections.connect("default", uri="./milvus_demo.db")
    else:
        connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)
    
    if utility.has_collection(COLLECTION_NAME):
        utility.drop_collection(COLLECTION_NAME)
        
    fields = [
        FieldSchema(name=ID_FIELD, dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name=VECTOR_FIELD, dtype=DataType.FLOAT_VECTOR, dim=EMBEDDING_DIM),
        FieldSchema(name=TEXT_FIELD, dtype=DataType.VARCHAR, max_length=65535)
    ]
    collection = Collection(name=COLLECTION_NAME, schema=CollectionSchema(fields))
    
    collection.insert([embeddings, chunks])
    print("正在写入磁盘 (Flush)...")
    collection.flush() 
    
    index_params = {"metric_type": "L2", "index_type": "IVF_FLAT", "params": {"nlist": 128}}
    collection.create_index(VECTOR_FIELD, index_params)
    collection.load()
    
    print(f"\n✅ 任务完成！共存入 {collection.num_entities} 条数据。")

if __name__ == "__main__":
    main()

正在加载文档...
   -> 正在移除页眉 (header) 和页脚 (footer)...
   -> 中间 HTML 文件已清理。
正在按段落切割 (合并目标: 500, 最小: 100, 硬上限: 1000)...
初步切割得到 9 个块，正在进行小块合并...
清理完成，最终得到 9 个文本块。

      最终切分效果预览      

>>> Chunk 0 (长度: 106)
内容: 〖特别报告〗 化解当下世界经济衰退的根本之道 ——基础设施建设 摘要：人类历史遭逢二战以来最为严重的经济衰退俨然成为定局，冲破至暗时刻化解全球经济衰退或将倚仗中...

>>> Chunk 1 (长度: 569)
内容: “此次经济衰退的范围和速度尚无现代先例，较之以往任何衰退都要严重得多。我们看到经济活动和就业率急剧下降，过去十年的经济成就已被倏然抹去……”美联储主席鲍威尔此番...

>>> Chunk 2 (长度: 459)
内容: 古语云“以史为鉴，方能知兴替”，人类经济史并不乏在遭遇经济萧条时冲破至暗时刻的先例，面对此轮史无前例的大衰退，亦可从历史发展的脉络中寻觅破局之道——基础设施建设...

>>> Chunk 3 (长度: 427)
内容: 二，日本基建狂潮成为经济托底神器。早在上世纪60年代，日本前首相田中角荣便提出“日本列岛改造论”，力主通过大规模建设全国新的铁路干线和高速公路来整顿交通网络，借...

>>> Chunk 4 (长度: 612)
内容: 三，“基建狂魔”创造中国式经济奇迹。就基建而言，中国可谓“青出于蓝”，是通过“大基建”让经济涅槃重生的历史践行者。1998年亚洲金融危机，中国经济遭受重创，出口...

... (及其他 4 个块)

正在生成向量...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it]


正在连接 Milvus (192.168.16.138)...
正在写入磁盘 (Flush)...

✅ 任务完成！共存入 9 条数据。
